In [1]:
// london_weather.csv : https://www.kaggle.com/datasets/emmanuelfwerr/london-weather-data
// Install the Microsoft.ML packages we're going to use for ML.NET
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.Data.Analysis"
#r "nuget:Microsoft.ML.TimeSeries"

Installed Packages Microsoft.Data.Analysis, 0.21.1 Microsoft.ML, 3.0.1 Microsoft.ML.TimeSeries, 3.0.1

Loading extensions from `C:\Users\ORHANT\.nuget\packages\microsoft.data.analysis\0.21.1\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

In [2]:
using Microsoft.ML;
using Microsoft.ML.Data;

In [3]:
public class WeatherInput
{
    [LoadColumn(0)]
    public DateTime WeatherDate; // Changed the Date format in source from: yyyyMMdd to: yyyy-MM-dd HH:mm:ss
    [LoadColumn(1)]
    public float CloudCover; // in oktas
    [LoadColumn(2)]
    public float Sunshine; // in hours
    [LoadColumn(3)]
    public float GlobalRads; // Global radiation in Watt / square meter
    [LoadColumn(4)]
    public float MaxTemp; // Celsius
    [LoadColumn(5)]
    public float MeanTemp; // Celsius
    [LoadColumn(6)]
    public float MinTemp; // Celsius
    [LoadColumn(7)]
    public float Precipitation; // This is the precipitation in millimeters. This is what we want to predict
    [LoadColumn(8)]
    public float Pressure; // In Pascals
    // Ignore snow depth since that won't be known at time of prediction and is related to Precipitation
}

In [4]:
// Create an ML Context. We'll use this for all ML operations
var context = new MLContext(seed: 2024);

In [5]:
// Load the data
var data = context.Data
.LoadFromTextFile<WeatherInput>("london_weather.csv"
, separatorChar: ','
, hasHeader: true
, allowQuoting: true);

In [6]:
// View schema
data.Schema

[ WeatherDate: DateTime, CloudCover: Single, Sunshine: Single, GlobalRads: Single, MaxTemp: Single, MeanTemp: Single, MinTemp: Single, Precipitation: Single, Pressure: Single ] Count 9 (values) index value 0 WeatherDate: DateTime Name WeatherDate Index 0 IsHidden False Type DateTime RawType System.DateTime Annotations Schema [ ] Count 0 (values) (empty) 1 CloudCover: Single Name CloudCover Index 1 IsHidden False Type Single RawType System.Single Annotations Schema [ ] Count 0 (values) (empty) 2 Sunshine: Single Name Sunshine Index 2 IsHidden False Type Single RawType System.Single Annotations Schema [ ] Count 0 (values) (empty) 3 GlobalRads: Single Name GlobalRads Index 3 IsHidden False Type Single RawType System.Single Annotations Schema [ ] Count 0 (values) (empty) 4 MaxTemp: Single Name MaxTemp Index 4 IsHidden False Type Single RawType System.Single Annotations Schema [ ] Count 0 (values) (empty) 5 MeanTemp: Single Name MeanTemp Index 5 IsHidden False Type Single RawType System.Single Annotations Schema [ ] Count 0 (values) (empty) 6 MinTemp: Single Name MinTemp Index 6 IsHidden False Type Single RawType System.Single Annotations Schema [ ] Count 0 (values) (empty) 7 Precipitation: Single Name Precipitation Index 7 IsHidden False Type Single RawType System.Single Annotations Schema [ ] Count 0 (values) (empty) 8 Pressure: Single Name Pressure Index 8 IsHidden False Type Single RawType System.Single Annotations Schema [ ] Count 0 (values) (empty)

In [7]:
// Preview data
data.Preview(3).RowView

index value 0 9 columns Values index value 0 [WeatherDate, 1/1/1979 12:00:00 AM] Key WeatherDate Value 1979-01-01 00:00:00Z 1 [CloudCover, 2] Key CloudCover Value 2 2 [Sunshine, 7] Key Sunshine Value 7 3 [GlobalRads, 52] Key GlobalRads Value 52 4 [MaxTemp, 2.3] Key MaxTemp Value 2.3 5 [MeanTemp, -4.1] Key MeanTemp Value -4.1 6 [MinTemp, -7.5] Key MinTemp Value -7.5 7 [Precipitation, 0.4] Key Precipitation Value 0.4 8 [Pressure, 101900] Key Pressure Value 101900 1 9 columns Values index value 0 [WeatherDate, 1/2/1979 12:00:00 AM] Key WeatherDate Value 1979-01-02 00:00:00Z 1 [CloudCover, 6] Key CloudCover Value 6 2 [Sunshine, 1.7] Key Sunshine Value 1.7 3 [GlobalRads, 27] Key GlobalRads Value 27 4 [MaxTemp, 1.6] Key MaxTemp Value 1.6 5 [MeanTemp, -2.6] Key MeanTemp Value -2.6 6 [MinTemp, -7.5] Key MinTemp Value -7.5 7 [Precipitation, 0] Key Precipitation Value 0 8 [Pressure, 102530] Key Pressure Value 102530 2 9 columns Values index value 0 [WeatherDate, 1/3/1979 12:00:00 AM] Key WeatherDate Value 1979-01-03 00:00:00Z 1 [CloudCover, 5] Key CloudCover Value 5 2 [Sunshine, 0] Key Sunshine Value 0 3 [GlobalRads, 13] Key GlobalRads Value 13 4 [MaxTemp, 1.3] Key MaxTemp Value 1.3 5 [MeanTemp, -2.8] Key MeanTemp Value -2.8 6 [MinTemp, -7.2] Key MinTemp Value -7.2 7 [Precipitation, 0] Key Precipitation Value 0 8 [Pressure, 102050] Key Pressure Value 102050

In [8]:
using Microsoft.Data.Analysis;
var df = data.ToDataFrame();

In [9]:
df.Head(5)

index,WeatherDate,CloudCover,Sunshine,GlobalRads,MaxTemp,MeanTemp,MinTemp,Precipitation,Pressure
0,1979-01-01 00:00:00Z,2,7,52,2.3,-4.1,-7.5,0.4,101900
1,1979-01-02 00:00:00Z,6,1.7,27,1.6,-2.6,-7.5,0,102530
2,1979-01-03 00:00:00Z,5,0,13,1.3,-2.8,-7.2,0,102050
3,1979-01-04 00:00:00Z,8,0,13,-0.3,-2.6,-6.5,0,100840
4,1979-01-05 00:00:00Z,6,2,29,5.6,-0.8,-1.4,0,102250


In [10]:
df.Info()

index,Info,WeatherDate,CloudCover,Sunshine,GlobalRads,MaxTemp,MeanTemp,MinTemp,Precipitation,Pressure
0,DataType,System.DateTime,System.Single,System.Single,System.Single,System.Single,System.Single,System.Single,System.Single,System.Single
1,Length (excluding null values),100,100,100,100,100,100,100,100,100


In [11]:
df.Description()

index,Description,WeatherDate,CloudCover,Sunshine,GlobalRads,MaxTemp,MeanTemp,MinTemp,Precipitation,Pressure
0,Length (excluding null values),100,100,100,100,100,100,100,100,100
1,Max,<null>,8,10.7,194,16.1,11.8,9.4,28.5,104270
2,Min,<null>,0,0,13,-0.8,-4.1,-7.5,0,98130
3,Mean,<null>,5.72,2.305,60.74,6.463999,3.1069994,-0.082999796,2.4940004,100905.4


In [12]:
// Split into train and test splits to detect overfitting
var split = context.Data.TrainTestSplit(data, testFraction: 0.2);

In [13]:
var testSet = split.TestSet; // 20 % of the data (0.2)

In [14]:
var trainSet = split.TrainSet; // 80 % of the data (0.8)

In [15]:
var featureArray = new string[] {"CloudCover","Sunshine","GlobalRads","MaxTemp","MeanTemp","MinTemp","Pressure"};

In [16]:
var processPipeline = context.Transforms.CopyColumns("Label", "Precipitation")
    .Append(context.Transforms.NormalizeMeanVariance("CloudCover", "CloudCover"))
    .Append(context.Transforms.NormalizeMeanVariance("Sunshine", "Sunshine"))
    .Append(context.Transforms.NormalizeMeanVariance("GlobalRads", "GlobalRads"))
    .Append(context.Transforms.NormalizeMeanVariance("MaxTemp", "MaxTemp"))
    .Append(context.Transforms.NormalizeMeanVariance("MeanTemp", "MeanTemp"))
    .Append(context.Transforms.NormalizeMeanVariance("MinTemp", "MinTemp"))
    .Append(context.Transforms.NormalizeMeanVariance("Pressure", "Pressure"))
    .Append(context.Transforms.Concatenate("Features", featureArray))
    .AppendCacheCheckpoint(context);

In [17]:
// Some of the available options with the base ML.NET package are: https://learn.microsoft.com/en-us/dotnet/api/microsoft.ml.trainers?view=ml-dotnet
//  * LbfgsPoissonRegression
//  * OnlineGradientDescent
//  * Sdca
// Select the algorithm we want
var trainer = context.Regression.Trainers.OnlineGradientDescent(labelColumnName: "Label", featureColumnName: "Features");

In [18]:
// Generate a training pipeline based on the processing pipeline mixed with the trainer
var trainingPipeline = processPipeline.Append(trainer);

In [19]:
// Train the model
var model = trainingPipeline.Fit(trainSet);

In [20]:
var testResults = model.Transform(testSet);
var testMetrics = context.Regression.Evaluate(testResults, "Label", "Score");
testMetrics

MeanAbsoluteError,2.118999537619063
MeanSquaredError,13.252040331744652
RootMeanSquaredError,3.6403351949710143
LossFunction,13.252040413413177
RSquared,0.06497927077698107


In [21]:
// Saving the Model
// context.Model.Save(model, data.Schema, "Model.zip")
// Loading the Model
// var (loadedModel, loadedSchema) = context.Model.Load("Model.zip");

In [22]:
public class WeatherPrediction
{
    [ColumnName("Score")]
    public float Precipitation; // This is the precipitation in millimeters. This is what we want to predict
}

In [23]:
var predictionEngine = context.Model.CreatePredictionEngine<WeatherInput, WeatherPrediction>(model);

In [24]:
var conditions = new WeatherInput {
    WeatherDate = DateTime.Now,
    CloudCover = 4f,
    Sunshine = 5.8f,
    GlobalRads = 50f,
    MaxTemp = 5.2f,
    MinTemp = 1.6f,
    MeanTemp = 3.7f,
    Pressure = 101170f,
    Precipitation = 0f, // Needed for the compiler, but its value is ignored
};

In [25]:
var prediction = predictionEngine.Predict(conditions);

In [26]:
prediction

Precipitation,1.0105166


In [27]:
public class WeatherInputTimeSeriesForecastedPrediction
{
    public float[] ForecastedPrecipitation { get; set; }
    public float[] ConfidenceLowerBound { get; set; }
    public float[] ConfidenceUpperBound { get; set; }
}

In [28]:
using Microsoft.ML.Transforms.TimeSeries;

In [29]:
var length = (int)trainSet.ToDataFrame().Rows.Count;
var forecastEstimator = context.Forecasting.ForecastBySsa(
    outputColumnName: nameof(WeatherInputTimeSeriesForecastedPrediction.ForecastedPrecipitation),
    inputColumnName: nameof(WeatherInput.Precipitation),
    windowSize: 12,
    seriesLength: length,
    trainSize: length,
    horizon: 4,
    confidenceLevel: 0.95f,
    confidenceLowerBoundColumn: nameof(WeatherInputTimeSeriesForecastedPrediction.ConfidenceLowerBound),
    confidenceUpperBoundColumn: nameof(WeatherInputTimeSeriesForecastedPrediction.ConfidenceUpperBound));

In [30]:
var forecastTransformerModel = forecastEstimator.Fit(trainSet);

In [31]:
var forecastingEngine = forecastTransformerModel.CreateTimeSeriesEngine<WeatherInput, WeatherInputTimeSeriesForecastedPrediction>(context);

In [32]:
var forecasts = forecastingEngine.Predict();

In [33]:
forecasts.Display()

ForecastedPrecipitation,"[ 2.9159963, 2.937026, 2.9426816, 2.9395862 ]"
ConfidenceLowerBound,"[ -5.3402095, -5.3204994, -5.316504, -5.3216367 ]"
ConfidenceUpperBound,"[ 11.172202, 11.194551, 11.201867, 11.200809 ]"
